# Avaliação Final Tecnologias Hacker - Análise de Logs Web

## Objetivo
O objetivo desse projeto é construir um código capaz de analisar se, de acordo com os logs, pode haver uma conexão suspeita no seu servidor web. Para isso, levamos em consideração que os logs disponibilizados estão em formato combined.

In [1]:
#bibliotecas
import re
from datetime import datetime


## Pré Processamento

In [2]:
#Função que le o arquivo e retorna uma lista com as linhas do arquivo
def ler_arquivo(nome_arquivo):
    try:
        with open(nome_arquivo, 'r') as arquivo:
            return arquivo.readlines()
    except FileNotFoundError:
        print("Arquivo não encontrado")
        return None

In [3]:
lista_linhas_log = ler_arquivo('logs/access.log')
lista_linhas_log[0]

'207.46.13.104 - - [08/Jul/2019:09:07:18 +0200] "GET /?_m=akcie&_c=3_rocnik_memorialu_romana_cunderlika HTTP/1.1" 302 623 "-" "Mozilla/5.0 (iPhone; CPU iPhone OS 7_0 like Mac OS X) AppleWebKit/537.51.1 (KHTML, like Gecko) Version/7.0 Mobile/11A465 Safari/9537.53 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)"\n'

In [4]:
# Função de extração das informações
def extrai_info_log(lista_linhas):
    info_logs = {}
    for i in range(len(lista_linhas)):
        dic_log = {}
        info = lista_linhas[i].split()
        dic_log['ip'] = info[0]
        dic_log['id_cliente'] = info[1]
        dic_log['usuario'] = info[2]
        dic_log['data'] = info[3].replace('[','')
        dic_log['requisicao'] = info[5] + " " + info[6] + " " + info[7]
        dic_log['status'] = info[8]
        dic_log['bytes_resposta'] = info[9]
        dic_log['referenciador'] = info[10]
        dic_log['navegador'] = info[11].replace('"','')

        info_logs[i] = dic_log

    return info_logs




In [5]:
dicionariodelogs = extrai_info_log(lista_linhas_log)
dicionariodelogs[0]

{'ip': '207.46.13.104',
 'id_cliente': '-',
 'usuario': '-',
 'data': '08/Jul/2019:09:07:18',
 'requisicao': '"GET /?_m=akcie&_c=3_rocnik_memorialu_romana_cunderlika HTTP/1.1"',
 'status': '302',
 'bytes_resposta': '623',
 'referenciador': '"-"',
 'navegador': 'Mozilla/5.0'}

In [6]:
#Função para separa somente os logs de get
def separa_logs_get(dicionario_logs):
    logs_get = {}
    for i in range(len(dicionario_logs)):
        if 'GET' in dicionario_logs[i]['requisicao']:
            logs_get[i] = dicionario_logs[i]
    return logs_get

#Função que separa os logs de post
def separa_logs_post(dicionario_logs):
    logs_post = {}
    for i in range(len(dicionario_logs)):
        if 'POST' in dicionario_logs[i]['requisicao']:
            logs_post[i] = dicionario_logs[i]
    return logs_post

In [7]:
dicionariodelogs_get = separa_logs_get(dicionariodelogs)
dicionariodelogs_post = separa_logs_post(dicionariodelogs)
dicionariodelogs_post

{1111: {'ip': '178.128.116.50',
  'id_cliente': '-',
  'usuario': '-',
  'data': '11/Jul/2019:00:52:50',
  'requisicao': '"POST /wp-admin/admin-ajax.php HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '526',
  'referenciador': '"-"',
  'navegador': 'Mozilla/5.0'},
 24062: {'ip': '89.108.87.179',
  'id_cliente': '-',
  'usuario': '-',
  'data': '22/Aug/2019:06:36:02',
  'requisicao': '"POST /xmlrpc.php HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '500',
  'referenciador': '"-"',
  'navegador': 'Mozilla/5.0'},
 26426: {'ip': '151.80.159.240',
  'id_cliente': '-',
  'usuario': '-',
  'data': '29/Aug/2019:01:56:05',
  'requisicao': '"POST /blog/xmlrpc.php HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '547',
  'referenciador': '"-"',
  'navegador': 'Mozilla/5.0'},
 26427: {'ip': '151.80.159.240',
  'id_cliente': '-',
  'usuario': '-',
  'data': '29/Aug/2019:01:56:05',
  'requisicao': '"POST /xmlrpc.php HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '537',
  'referenciador'

### Analisando os logs contendo POST


**coisas suspeitas com POST**:
- post em caminhos suspeitos (/wp-login.php, /xmlrpc.php, ou APIs internas)

- muitas requisições em um curto intervalo

- sql injection (' OR '1'='1, UNION SELECT, ou DROP TABLE.)

- tentativas de login/senhas diferentes: ataque brute force (não sei se o gpt colocou mas eh sobre)

- tentativa de enviar dados com script (script, onload=, ou eval( em campos de texto (indício de XSS)))

- requisições com corpo muito maior que o esperado

- headers http incomuns (X-Forwarded-For alterado ou user-agent indicando automação)

- Arquivos com extensões como .php, .exe, .jsp, ou .sh

- Detecção de endpoints que deveriam ser privados, mas estão sendo acessados publicamente

- Dados contendo comandos como ; ls, && whoami, ou | netstat.

In [8]:
def caminhos_suspeitos(log_post):
    logs_suspeitos = {}
    padroes_suspeitos = [
        # WordPress e CMS
        'wp-login', 'wp-admin', 'xmlrpc.php', 'admin-ajax.php', 
        'wp-content', 'wp-content/plugins/', 'wp-includes/', 'wp-config.php', 'wp-json/',
        # Painéis de Administração
        '/admin', '/administrator', '/console', '/controlpanel', '/login',
        '/dashboard', '/dbadmin', '/config', '/config.php', '/admin.php',
        # Arquivos de Configuração e Backup
        '.env', 'config.php', 'settings.php', 'backup.sql', 'db_backup.sql',
        'db_dump.sql', '.htaccess', '.htpasswd',
        # Caminhos Sensíveis ou de Sistema
        '/etc/passwd', '/etc/shadow', '/proc/self/environ', '/var/www/html',
        # APIs Vulneráveis
        '/api/v1/', '/graphql', '/admin/api/',
        # Exploração de Ferramentas e Bibliotecas
        '/phpmyadmin', '/server-status', '/actuator/health',
        # Frameworks e Ferramentas
        '/debug', '/shell', '/cmd',
        # Padrões de Ataques
        '?file=', '?path=', '?cmd=', '?exec=', '?debug=', '?action=',
        # Explorações de Vulnerabilidades
        'cgi-bin/', 'shell.php', 'eval-stdin.php',
        # Padrões Genéricos de Exploração (LFI)
        '../', '.php', '.sh', '.exe'
    ]

    for key, value in log_post.items():
        for padrao in padroes_suspeitos:
            if padrao in value['requisicao']:
                motivo = f"Padrão suspeito: {padrao}"
                value['motivo'] = [motivo]
                logs_suspeitos[key] = value
                break 
    return logs_suspeitos


In [9]:
logs_post_caminhos_suspeitos = caminhos_suspeitos(dicionariodelogs_post)
logs_post_caminhos_suspeitos

{1111: {'ip': '178.128.116.50',
  'id_cliente': '-',
  'usuario': '-',
  'data': '11/Jul/2019:00:52:50',
  'requisicao': '"POST /wp-admin/admin-ajax.php HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '526',
  'referenciador': '"-"',
  'navegador': 'Mozilla/5.0',
  'motivo': ['Padrão suspeito: wp-admin']},
 24062: {'ip': '89.108.87.179',
  'id_cliente': '-',
  'usuario': '-',
  'data': '22/Aug/2019:06:36:02',
  'requisicao': '"POST /xmlrpc.php HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '500',
  'referenciador': '"-"',
  'navegador': 'Mozilla/5.0',
  'motivo': ['Padrão suspeito: xmlrpc.php']},
 26426: {'ip': '151.80.159.240',
  'id_cliente': '-',
  'usuario': '-',
  'data': '29/Aug/2019:01:56:05',
  'requisicao': '"POST /blog/xmlrpc.php HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '547',
  'referenciador': '"-"',
  'navegador': 'Mozilla/5.0',
  'motivo': ['Padrão suspeito: xmlrpc.php']},
 26427: {'ip': '151.80.159.240',
  'id_cliente': '-',
  'usuario': '-',
  'data': '

In [10]:
def checando_sql_injection(logs_post):
    logs_suspeitos = {}
    padroes_suspeitos = [
        'select', 'union', 'insert', 'update', 'delete', 'drop', 'alter', 'create',
        'truncate', 'exec', 'grant', 'revoke',
        "' or '1'='1", "--", "#", ";", "/*", "*/",
        "char(", "concat(", "load_file(", "sleep(",
        "information_schema", "table_schema", "column_name",
        "'=", "like '%'", "and 1=1"
    ]
    
    for key, value in logs_post.items():
        for padrao in padroes_suspeitos:
            if padrao in value['requisicao'].lower():
                motivo = f"Padrão suspeito: {padrao}"
                value['motivo'] = [motivo]
                logs_suspeitos[key] = value
                break 
    return logs_suspeitos

In [11]:
logs_sql_injection = checando_sql_injection(dicionariodelogs_post)
logs_sql_injection

{139274: {'ip': '93.99.104.182',
  'id_cliente': '-',
  'usuario': '-',
  'data': '12/May/2020:01:15:24',
  'requisicao': '"POST /?_m=studium&amp%3B_c=zdravotnicky_zachranar&ABBv%3D5712%20AND%201%3D1%20UNION%20ALL%20SELECT%201%2CNULL%2C%27%3Cscript%3Ealert%28%22XSS%22%29%3C%2Fscript%3E%27%2Ctable_name%20FROM%20information_schema.tables%20WHERE%202%3E1--%2F%2A%2A%2F%3B%20EXEC%20xp_cmdshell%28%27cat%20..%2F..%2F..%2Fetc%2Fpasswd%27%29%23 HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '1106',
  'referenciador': '"-"',
  'navegador': 'Mozilla/5.0',
  'motivo': ['Padrão suspeito: select']},
 139280: {'ip': '93.99.104.182',
  'id_cliente': '-',
  'usuario': '-',
  'data': '12/May/2020:01:15:30',
  'requisicao': '"POST /?_m=studium%26amp%3B_c%3Dzdravotnicky_zachranar%29%20AND%20%28SELECT%205794%20FROM%28SELECT%20COUNT%28%2A%29%2CCONCAT%280x716b6a7a71%2C%28SELECT%20%28ELT%285794%3D5794%2C1%29%29%29%2C0x716b707671%2CFLOOR%28RAND%280%29%2A2%29%29x%20FROM%20INFORMATION_SCHEMA.PLUGINS%20GROUP%

In [12]:
#Ataque XSS
def enviar_dados_script(logs_post):
    logs_suspeitos = {}
    padroes_suspeitos = [
        '<script>', 'onload=', 'eval(', 
        '<iframe>', 'javascript:', '<embed>', '<object>', '<applet>',
        'onclick=', 'onmouseover=', 'onerror=',
        '%3Cscript%3E', '%3Ciframe%3E',
        'alert(', 'document.cookie', 'window.location', 'innerHTML',
        '"></script>', '<img src="x" onerror=', '"><svg onload=',
        '{{7*7}}', '${7*7}', 
        'base64', '\\u'
    ]

    for key, value in logs_post.items():
        for padrao in padroes_suspeitos:
            if padrao in value['requisicao'].lower():
                motivo = f"Padrão suspeito: {padrao}"
                value['motivo'] = [motivo]
                logs_suspeitos[key] = value
                break
    return logs_suspeitos
    

In [13]:
logs_dados_script = enviar_dados_script(dicionariodelogs_post)
logs_dados_script

{257057: {'ip': '222.186.30.214',
  'id_cliente': '-',
  'usuario': '-',
  'data': '02/Apr/2021:18:14:57',
  'requisicao': '"GET /uploads/dede/sys_verifies.php?action=getfiles&refiles%5B0%5D=123&refiles%5B1%5D=%5C%22;eval($_POST%5Bysy%5D);die();// HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '758',
  'referenciador': '"http://szsbb.eu/uploads/dede/sys_verifies.php?action=getfiles&refiles[0]=123&refiles[1]=\\\\%22;eval($_POST[ysy]);die();//"',
  'navegador': 'Mozilla/5.0',
  'motivo': ['Padrão suspeito: eval(']},
 257058: {'ip': '222.186.30.214',
  'id_cliente': '-',
  'usuario': '-',
  'data': '02/Apr/2021:18:14:57',
  'requisicao': '"GET /uploads/dede/sys_verifies.php?action=getfiles&refiles%5B0%5D=123&refiles%5B1%5D=%5C%22;eval($_POST%5Bysy%5D);die();// HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '758',
  'referenciador': '"http://szsbb.eu/uploads/dede/sys_verifies.php?action=getfiles&refiles[0]=123&refiles[1]=\\\\%22;eval($_POST[ysy]);die();//"',
  'navegador': 'Mozilla

In [14]:
#DDos
def detectar_frequencia_acessos(logs_post, intervalo_segundos=10, limite_requisicoes=5):
    logs_suspeitos = {}
    ip_atividades = {}

    for key, value in logs_post.items():
        ip = value['ip']
        data = datetime.strptime(value['data'], '%d/%b/%Y:%H:%M:%S')

        if ip not in ip_atividades:
            ip_atividades[ip] = []
        ip_atividades[ip].append((key, data))

    for ip, acessos in ip_atividades.items():
        acessos.sort(key=lambda x: x[1])  

        for i in range(len(acessos) - limite_requisicoes + 1):
            inicio = acessos[i][1]
            fim = acessos[i + limite_requisicoes - 1][1]
            delta = (fim - inicio).total_seconds()

            if delta <= intervalo_segundos:
                
                for j in range(i, i + limite_requisicoes):
                    key = acessos[j][0]
                    motivo = f"Alta frequência de acessos: {limite_requisicoes} requisições em {delta} segundos"
                    logs_post[key]['motivo'] = [motivo]
                    logs_suspeitos[key] = logs_post[key]
                break

    return logs_suspeitos

In [15]:
logs_frequencia_acessos = detectar_frequencia_acessos(dicionariodelogs_post, intervalo_segundos=10, limite_requisicoes=5)
logs_frequencia_acessos

{139273: {'ip': '93.99.104.182',
  'id_cliente': '-',
  'usuario': '-',
  'data': '12/May/2020:01:15:23',
  'requisicao': '"POST /?_m=studium&amp%3B_c=zdravotnicky_zachranar HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '570',
  'referenciador': '"-"',
  'navegador': 'Mozilla/5.0',
  'motivo': ['Alta frequência de acessos: 5 requisições em 5.0 segundos']},
 139274: {'ip': '93.99.104.182',
  'id_cliente': '-',
  'usuario': '-',
  'data': '12/May/2020:01:15:24',
  'requisicao': '"POST /?_m=studium&amp%3B_c=zdravotnicky_zachranar&ABBv%3D5712%20AND%201%3D1%20UNION%20ALL%20SELECT%201%2CNULL%2C%27%3Cscript%3Ealert%28%22XSS%22%29%3C%2Fscript%3E%27%2Ctable_name%20FROM%20information_schema.tables%20WHERE%202%3E1--%2F%2A%2A%2F%3B%20EXEC%20xp_cmdshell%28%27cat%20..%2F..%2F..%2Fetc%2Fpasswd%27%29%23 HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '1106',
  'referenciador': '"-"',
  'navegador': 'Mozilla/5.0',
  'motivo': ['Alta frequência de acessos: 5 requisições em 5.0 segundos']},
 1392

In [16]:
def verificar_status_suspeitos(logs_post):
    logs_suspeitos = {}
    status_suspeitos=['401', '403', '404', '500']

    for key, value in logs_post.items():
        #print(value['status'])
        if value['status'] in status_suspeitos:
            motivo = f"Código de status suspeito: {value['status']}"
            value['motivo'] = [motivo]
            logs_suspeitos[key] = value

    return logs_suspeitos


In [17]:
logs_status_suspeitos = verificar_status_suspeitos(dicionariodelogs_post)
logs_status_suspeitos

{}

In [18]:
def detectar_mudanca_bytes(logs_post):
    logs_suspeitos = {}
    requisicoes_analisadas = {}

    for key, value in logs_post.items():
        requisicao = value['requisicao']
        bytes_resposta = int(value['bytes_resposta'])

        if requisicao not in requisicoes_analisadas:
            requisicoes_analisadas[requisicao] = []
        else:
            tamanhos = requisicoes_analisadas[requisicao]
            if bytes_resposta not in tamanhos:
                motivo = f"Mudança no tamanho da resposta para a mesma requisição: {requisicao}"
                value['motivo'] = [motivo]
                logs_suspeitos[key] = value
        requisicoes_analisadas[requisicao].append(bytes_resposta)

    return logs_suspeitos


In [19]:
logs_mudanca_bytes = detectar_mudanca_bytes(dicionariodelogs_post)
logs_mudanca_bytes

{26427: {'ip': '151.80.159.240',
  'id_cliente': '-',
  'usuario': '-',
  'data': '29/Aug/2019:01:56:05',
  'requisicao': '"POST /xmlrpc.php HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '537',
  'referenciador': '"-"',
  'navegador': 'Mozilla/5.0',
  'motivo': ['Mudança no tamanho da resposta para a mesma requisição: "POST /xmlrpc.php HTTP/1.1"']},
 43416: {'ip': '106.54.217.47',
  'id_cliente': '-',
  'usuario': '-',
  'data': '07/Oct/2019:02:24:33',
  'requisicao': '"POST / HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '484',
  'referenciador': '"-"',
  'navegador': '-',
  'motivo': ['Mudança no tamanho da resposta para a mesma requisição: "POST / HTTP/1.1"']},
 53950: {'ip': '201.144.43.197',
  'id_cliente': '-',
  'usuario': '-',
  'data': '26/Oct/2019:10:07:10',
  'requisicao': '"POST /xmlrpc.php HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '541',
  'referenciador': '"-"',
  'navegador': 'Mozilla/5.0',
  'motivo': ['Mudança no tamanho da resposta para a mesma requ

In [20]:
def verificar_referenciador_navegador(logs_post):
    logs_suspeitos = {}
    navegador_nao_suspeitos=['Mozilla', 'Chrome', 'Safari', 'Opera', 'Edge', 'Trident']

    for key, value in logs_post.items():
        #if value['referenciador'] == '"-"':
        #    motivo = "Referenciador ausente ou suspeito"
        #    value['motivo'] = [motivo]
        #    logs_suspeitos[key] = value
        if value['navegador'].split('/')[0] not in navegador_nao_suspeitos:
            motivo = "Navegador suspeito ou desconhecido"
            value['motivo'] = [motivo]
            logs_suspeitos[key] = value

    return logs_suspeitos


In [50]:
logs_referenciador_navegador = verificar_referenciador_navegador(dicionariodelogs_post)
logs_referenciador_navegador

{43416: {'ip': '106.54.217.47',
  'id_cliente': '-',
  'usuario': '-',
  'data': '07/Oct/2019:02:24:33',
  'requisicao': '"POST / HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '484',
  'referenciador': '"-"',
  'navegador': '-',
  'motivo': ['Navegador suspeito ou desconhecido']},
 79556: {'ip': '49.234.7.207',
  'id_cliente': '-',
  'usuario': '-',
  'data': '15/Dec/2019:21:39:43',
  'requisicao': '"POST / HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '484',
  'referenciador': '"-"',
  'navegador': '-',
  'motivo': ['Navegador suspeito ou desconhecido']},
 83737: {'ip': '173.231.63.87',
  'id_cliente': '-',
  'usuario': '-',
  'data': '28/Dec/2019:01:04:40',
  'requisicao': '"POST / HTTP/1.1"',
  'status': '302',
  'bytes_resposta': '480',
  'referenciador': '"-"',
  'navegador': 'Go-http-client/1.1',
  'motivo': ['Navegador suspeito ou desconhecido']},
 112075: {'ip': '129.226.115.132',
  'id_cliente': '-',
  'usuario': '-',
  'data': '03/Mar/2020:18:05:16',
  'requisicao':

# Montando o relatório final